<a href="https://colab.research.google.com/github/kattens/PubChem-Data-Handler/blob/main/Pubchem_Downloader_Phase_one.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PubChemPy

  Preparing metadata (setup.py) ... done
  Created wheel for PubChemPy: filename=PubChemPy-1.0.4-py3-none-any.whl size=13819 sha256=bcf00d6cac6a608ad2f19eed84db541c1a9273643a0983c9b75c2b425330b12a
  Stored in directory: /root/.cache/pip/wheels/90/7c/45/18a0671e3c3316966ef7ed9ad2b3f3300a7e41d3421a44e799
Successfully built PubChemPy


In [ ]:
import pubchempy as pcp
import csv

def fetch_bioassay_results(pubchem_ids):
    results = []
    for cid in pubchem_ids:
        try:
            compound = pcp.Compound.from_cid(cid)
            assays = compound.get_assays()
            for assay in assays:
                result = {
                    'CID': cid,
                    'Assay ID': assay.aid,
                    'Description': assay.description,
                    'Results': str(assay.results)  # Convert results to string if needed
                }
                results.append(result)
        except Exception as e:
            results.append({'CID': cid, 'Error': str(e)})
    return results

def save_results_to_csv(results, filename):
    keys = results[0].keys()
    with open(filename, 'w', newline='') as output_file:
        dict_writer = csv.DictWriter(output_file, keys)
        dict_writer.writeheader()
        dict_writer.writerows(results)

# List of PubChem CIDs
pubchem_ids = [5330175]  # Replace these with your actual CIDs

# Fetching the bioassay results
bioassay_results = fetch_bioassay_results(pubchem_ids)

# Save the results to a CSV file
save_results_to_csv(bioassay_results, 'bioassay_results.csv')

